In [1]:
#Reference Code: https://github.com/aimacode/aima-python/blob/master/search.py

from queue import Queue
import copy
import time 

In [2]:
class Problem:
    
    def __init__(self, initial, goal=None):            #This constructor specifies the initial state
        self.initial = initial
        
    def domain_values(self, numbers, used):         #it returns the domain values
        return [number for number in numbers if number not in used]
    
    def find_empty(self, puzzle, state):            #Returns the empty position i.e, position with '0'
        for row in range(puzzle):
            for column in range(puzzle):
                if state[row][column] == 0:
                    return row, column
                
    def result(self, state, action):                   # Returns updated state after adding new valid value
        play = action[0]
        row = action[1]
        column = action[2]

        new_state = copy.deepcopy(state)         #This adds new valid value to the state, without repeating previous ones
        new_state[row][column] = play

        return new_state
    

    def goal_test(self, state):                #Tests the state and return true if reached to goal

        total = 45    #The sum of each row, colum, and small grid (quadrant)

        # Check rows and columns and return false if total is invalid
        for row in range(9):
            if (len(state[row]) != 9) or (sum(state[row]) != total):
                return False

            column_total = 0
            for column in range(9):
                column_total += state[column][row]

            if (column_total != total):
                return False

        # Check quadrants and return false if total is invalid
        for column in range(0,9,3):
            for row in range(0,9,3):

                grid_total = 0
                for grid_row in range(0,3):
                    for grid_column in range(0,3):
                        grid_total += state[row + grid_row][column + grid_column]

                if (grid_total != total):
                    return False
        return True
    
    
    def constraints(self, state):
        number_set = range(1, 10) #the set of values whixh can be put on board
        in_column = [] #the list of vaid values for column
        in_block = [] #the list of vaid values for block or quadrant

        row,column = self.find_empty(9, state) #get the first empty position 

        # Filter valid values based on row
        in_row = [number for number in state[row] if (number != 0)]
        options = self.domain_values(number_set, in_row)

        # Filter valid values based on column
        for column_index in range(9):
            if state[column_index][column] != 0:
                in_column.append(state[column_index][column])
        options = self.domain_values(options, in_column)

        # Filter with valid values based on block or quadrant
        row_start = int(row/3)*3
        column_start = int(column/3)*3
        
        for block_row in range(0, 3):
            for block_column in range(0,3):
                in_block.append(state[row_start + block_row][column_start + block_column])
        options = self.domain_values(options, in_block)

        for number in options:
            yield number, row, column

In [3]:
class Node:

    def __init__(self, state, action=None):   #This constructor creates a search tree with state and action 
        self.state = state
        self.action = action

    def expand(self, problem):   #Expand the problem and return list of nodes reachable in one step from this node
        return [self.child_node(problem, action) for action in problem.constraints(self.state)]

    def child_node(self, problem, action):   #return a node with new state
        next = problem.result(self.state, action)
        return Node(next, action)

In [4]:
def BFS(problem, node):

    if problem.goal_test(node.state):   #check if already at goal state
        return node

    temp_solution = Queue()         #FIFO queue for BFS
    temp_solution.put(node)       

    while (temp_solution.qsize() != 0):   # Loop until all nodes are explored or solution found

        node = temp_solution.get()
        for child_node in node.expand(problem):
            if problem.goal_test(child_node.state):
                return child_node

            temp_solution.put(child_node)

    return None

In [ ]:
no =1
file = open("testData.txt","r")
start = time.time()                 #recording the start time
for line in file:
    line=line.strip()
    example=line.replace('.','0')
    s=[]
    for i in example:
        s.append(int(i))
    puzzle=[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]
           ,[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
    index=0
    for row in range(9):
        for column in range(9):
            puzzle[row][column] = s[index]
            index+=1
    tree = Problem(puzzle)
    root_node = Node(tree.initial)
    solution = BFS(tree, root_node)
    if solution:
        print ("Found solution: ",no)
        no+=1
           # for row in solution.state:
             #   print (row)
    else:
        print ("No possible solutions")
file.close()
elapsed_time = time.time() - start
print ("Elapsed time: " + str(elapsed_time))